# Generate Satellite Images ID from AOI and Date Range

Get satellite image ids for download from Planet API

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from requests.auth import HTTPBasicAuth
from multiprocessing.dummy import Pool as ThreadPool

from planet_utils import get_datetime, get_save_search
from pathum_filter import adm1_pathum_bbox_df as pathum_geometry
from pathum_filter import get_pathum_filter, get_stats, get_ids

In [2]:
## User Params
item_type = "PSOrthoTile"
asset_type = "visual"
img_folder = './data'
area_name = 'pathum'

start_date = get_datetime(2019, 1, 1)
end_date = get_datetime(2019, 11, 1)

img_id_file = area_name+'_'+start_date[:10]+'_'+end_date[:10]+'.txt'
'Output file = ' + img_id_file

'Output file = pathum_2019-01-01_2019-11-01.txt'

Show AOI (Area of Interest)

In [3]:
from geojsonio import display
display(pathum_geometry.to_json())

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22id%22%3A%20%220%22%2C%20%22type%22%3A%20%22Feature%22%2C%20%22properties%22%3A%20%7B%7D%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B100.33162488100004%2C%2013.916254426000023%5D%2C%20%5B100.95184798700006%2C%2013.916254426000023%5D%2C%20%5B100.95184798700006%2C%2014.27595109400005%5D%2C%20%5B100.33162488100004%2C%2014.27595109400005%5D%2C%20%5B100.33162488100004%2C%2013.916254426000023%5D%5D%5D%7D%7D%5D%7D'

In [4]:
#%% get number of available images
# aoi is the bounding box of Pathum Thani
aoi_geo = pathum_geometry.__geo_interface__['features'][0]['geometry']
pathum_filter = get_pathum_filter(start_date, end_date, aoi_geo)
result = get_stats(pathum_filter)
print(result)

# convert to DataFrame for tidiness
df = pd.DataFrame.from_dict(result.json()['buckets'])
df.set_index('start_time', inplace=True)
df.index = pd.to_datetime(df.index)
df.head()

<Response [200]>


,count
start_time,
2019-01-01 00:00:00+00:00,11
2019-01-02 00:00:00+00:00,4
2019-01-03 00:00:00+00:00,0
2019-01-04 00:00:00+00:00,0
2019-01-05 00:00:00+00:00,0


Show number of images to download

In [5]:
import pandas_bokeh
pandas_bokeh.output_notebook()
p = df.plot_bokeh(kind="barh", hovertool=False, show_figure=False)
p.xaxis.axis_label = 'Number of Images'
p.yaxis.axis_label = 'Acquired Date'
p.title.text = f'Pathum Thani: Total {df["count"].sum()} Image from {start_date[:10]} to {end_date[:10]}'

from bokeh.models import HoverTool
h = HoverTool(mode="hline")
h.tooltips = [('Acquired date', '@__x__values_original{%Y-%m-%d}'), ('No. of imgs', '@count{%d}')]
#["@__x__values_original{%Y-%m-%d %H:%M:%S}"
h.formatters = {'__x__values_original': 'datetime','count':'printf'}
#{"__x__values_original": "datetime"}
p.add_tools(h)
pandas_bokeh.show(p)

Loading BokehJS ...

Get search ID & image item ID and write into the text file.

In [6]:
if os.path.isfile(img_id_file):
    print('File existed, Do Nothing')
else:
    search_id, img_ids = get_save_search(pathum_filter)
    
    with open(img_id_file, 'w') as f:
        f.write("%s\n" % search_id)
        f.write("%s\n" % item_type)
        f.write("%s\n" % asset_type)
        for item in img_ids:
            f.write("%s\n" % item)
    print('Image IDs Saved')

<Response [200]>
Page 0: 100 images
Page 1: 100 images
Page 2: 100 images
Page 3: 100 images
Page 4: 100 images
Page 5: 100 images
Page 6: 100 images
Page 7: 100 images
Page 8: 100 images
Page 9: 100 images
Page 10: 100 images
Page 11: 100 images
Page 12: 100 images
Page 13: 100 images
Page 14: 100 images
Page 15: 100 images
Page 16: 100 images
Page 17: 100 images
Page 18: 100 images
Page 19: 41 images
Page 20: 0 images
Image IDs Saved
